# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(131)
initial_params = np.random.random([105])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 1, 2, 3]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[0, 1])
    ArbitraryUnitary(params[15:30], wires=[2, 3])
    ArbitraryUnitary(params[30:45], wires=[4, 5])
    ArbitraryUnitary(params[45:60], wires=[6, 7])

    # Second layer
    ArbitraryUnitary(params[60:75], wires=[1, 2])
    ArbitraryUnitary(params[75:90], wires=[5, 6])

    # Third layer
    ArbitraryUnitary(params[90:105], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [10]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057  0.82111597 0.98499188 0.93449492
 0.90305486 0.3380262  0.78324429 0.74373474 0.58058546 0.43266356
 0.66792795 0.23668741 0.45173663 0.91999741 0.96687301 0.76905057
 0.32671177 0.62283984 0.19160224 0.24832171 0.11683869 0.01032549
 0.84869439 0.53066622 0.49383494 0.65230764 0.22667618 0.61716246
 0.2005521  0.49093199 0.94852991 0.98197027 0.17315162 0.74251972
 0.38646803 0.65110712 0.7434636  0.24777039 0.49229597 0.42474983
 0.29006936 0.98668888 0.33945016 0.1997036

# Accuracy test definition

In [11]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [12]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.487875150060024
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.6055222088835535
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.637454981992797
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.6989195678271308
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.6979591836734693
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.690516206482593
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.6883553421368547
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.7334933973589436
best accuracy so far!
epoch 71
epoch 72
epoch 73
epoch

epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7286914765906363
test_accuracies: [0.6883553421368547, 0.7334933973589436, 0.7325330132052821, 0.7303721488595438, 0.7243697478991596, 0.7183673469387755, 0.7428571428571429, 0.7085234093637455, 0.7289315726290516, 0.7174069627851141, 0.714765906362545, 0.7315726290516207, 0.726530612244898, 0.7039615846338536, 0.7243697478991596, 0.7320528211284514, 0.7020408163265306, 0.7164465786314526, 0.7466986794717887, 0.7390156062424971, 0.7354141656662665, 0.7361344537815127, 0.6912364945978391, 0.7284513805522208, 0.7150060024009603, 0.7337334933973589, 0.7186074429771909, 0.7298919567827131, 0.7234093637454981, 0.7286914765906363]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7517406962785114
best accuracy so far!
test_accuracies: [0.7334933973589436, 0.7325330132052821, 0.7303721488595438, 0.7243697478991596, 0.7183673469387755, 0.7428571428571429, 0.70852340936374

epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.7438175270108043
test_accuracies: [0.7315726290516207, 0.726530612244898, 0.7039615846338536, 0.7243697478991596, 0.7320528211284514, 0.7020408163265306, 0.7164465786314526, 0.7466986794717887, 0.7390156062424971, 0.7354141656662665, 0.7361344537815127, 0.6912364945978391, 0.7284513805522208, 0.7150060024009603, 0.7337334933973589, 0.7186074429771909, 0.7298919567827131, 0.7234093637454981, 0.7286914765906363, 0.7517406962785114, 0.7351740696278511, 0.7176470588235294, 0.719327731092437, 0.7392557022809123, 0.7464585834333733, 0.7406962785114046, 0.7207683073229292, 0.7430972388955582, 0.7409363745498199, 0.7438175270108043]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7539015606242497
best accuracy so far!
test_accuracies: [0.726530612244898, 0.7039615846338536, 0.7243697478991596, 0.7320528211284514, 0.7020408163265306, 0.7164465786314526, 0.74669

epoch 563
epoch 564
epoch 565
epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.7493397358943578
test_accuracies: [0.6912364945978391, 0.7284513805522208, 0.7150060024009603, 0.7337334933973589, 0.7186074429771909, 0.7298919567827131, 0.7234093637454981, 0.7286914765906363, 0.7517406962785114, 0.7351740696278511, 0.7176470588235294, 0.719327731092437, 0.7392557022809123, 0.7464585834333733, 0.7406962785114046, 0.7207683073229292, 0.7430972388955582, 0.7409363745498199, 0.7438175270108043, 0.7539015606242497, 0.6713085234093638, 0.7421368547418967, 0.7255702280912365, 0.715486194477791, 0.7361344537815127, 0.7250900360144057, 0.7476590636254502, 0.7066026410564226, 0.7476590636254502, 0.7493397358943578]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.7301320528211285
test_accuracies: [0.7284513805522208, 0.7150060024009603, 0.7337334933973589, 0.7186074429771909, 0.7298919567827131, 0.7234093637454981, 0.728691

epoch 674
epoch 675
epoch 676
epoch 677
epoch 678
epoch 679
epoch 680
accuracy: 0.7438175270108043
test_accuracies: [0.719327731092437, 0.7392557022809123, 0.7464585834333733, 0.7406962785114046, 0.7207683073229292, 0.7430972388955582, 0.7409363745498199, 0.7438175270108043, 0.7539015606242497, 0.6713085234093638, 0.7421368547418967, 0.7255702280912365, 0.715486194477791, 0.7361344537815127, 0.7250900360144057, 0.7476590636254502, 0.7066026410564226, 0.7476590636254502, 0.7493397358943578, 0.7301320528211285, 0.7214885954381753, 0.719327731092437, 0.7380552220888356, 0.7414165666266507, 0.7344537815126051, 0.7258103241296519, 0.7450180072028811, 0.7380552220888356, 0.7296518607442977, 0.7438175270108043]
epoch 681
epoch 682
epoch 683
epoch 684
epoch 685
epoch 686
epoch 687
epoch 688
epoch 689
epoch 690
accuracy: 0.7330132052821128
test_accuracies: [0.7392557022809123, 0.7464585834333733, 0.7406962785114046, 0.7207683073229292, 0.7430972388955582, 0.7409363745498199, 0.7438175270108043,

epoch 784
epoch 785
epoch 786
epoch 787
epoch 788
epoch 789
epoch 790
accuracy: 0.7527010804321729
test_accuracies: [0.7255702280912365, 0.715486194477791, 0.7361344537815127, 0.7250900360144057, 0.7476590636254502, 0.7066026410564226, 0.7476590636254502, 0.7493397358943578, 0.7301320528211285, 0.7214885954381753, 0.719327731092437, 0.7380552220888356, 0.7414165666266507, 0.7344537815126051, 0.7258103241296519, 0.7450180072028811, 0.7380552220888356, 0.7296518607442977, 0.7438175270108043, 0.7330132052821128, 0.7627851140456182, 0.7510204081632653, 0.756062424969988, 0.7476590636254502, 0.7454981992797118, 0.709483793517407, 0.7130852340936374, 0.731812725090036, 0.7563025210084033, 0.7527010804321729]
epoch 791
epoch 792
epoch 793
epoch 794
epoch 795
epoch 796
epoch 797
epoch 798
epoch 799
epoch 800
accuracy: 0.741656662665066
test_accuracies: [0.715486194477791, 0.7361344537815127, 0.7250900360144057, 0.7476590636254502, 0.7066026410564226, 0.7476590636254502, 0.7493397358943578, 0.7

epoch 898
epoch 899
epoch 900
accuracy: 0.7181272509003602
test_accuracies: [0.7380552220888356, 0.7414165666266507, 0.7344537815126051, 0.7258103241296519, 0.7450180072028811, 0.7380552220888356, 0.7296518607442977, 0.7438175270108043, 0.7330132052821128, 0.7627851140456182, 0.7510204081632653, 0.756062424969988, 0.7476590636254502, 0.7454981992797118, 0.709483793517407, 0.7130852340936374, 0.731812725090036, 0.7563025210084033, 0.7527010804321729, 0.741656662665066, 0.7582232893157264, 0.7375750300120048, 0.7553421368547419, 0.7430972388955582, 0.7438175270108043, 0.7555822328931573, 0.756782713085234, 0.7481392557022809, 0.7397358943577431, 0.7181272509003602]
epoch 901
epoch 902
epoch 903
epoch 904
epoch 905
epoch 906
epoch 907
epoch 908
epoch 909
epoch 910
accuracy: 0.7503001200480193
test_accuracies: [0.7414165666266507, 0.7344537815126051, 0.7258103241296519, 0.7450180072028811, 0.7380552220888356, 0.7296518607442977, 0.7438175270108043, 0.7330132052821128, 0.7627851140456182, 0

In [13]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 0.59604012  0.92024838 -0.55786511  1.57708124  1.41633609  0.3124529
 -0.18559679  0.40305999  0.25355608  1.13050408  0.64832521 -0.73699256
  1.22984825  0.32446546  0.77531161  0.63126438  0.71704114 -0.70293377
  0.59546525  0.06183586  1.50862841  1.21394985  0.63664821  0.94102745
  1.42467009  0.02725095  0.5824784   1.11204753  0.31866728  0.23132314
  2.0909241   0.09767428  0.95475314  0.48296839  2.79441395 -0.4525607
  1.58727576  0.18620613  0.66141621  0.57556623  1.21180394  1.48858783
  0.92146493 -0.90439925  0.6408585   1.69201751 -1.52717579  0.39491794
  0.35677768  1.74712519  1.34393982  0.93644295  1.74404995 -0.0135665
  0.02380869 -0.27721111  1.14829096  1.31689617  1.31836154  0.76905057
  1.18818097  0.06539514  0.08077096  0.21867097  0.51487058  0.51517001
  0.53914976  0.67758194  0.39870639  0.79671153 -0.29858772  1.3998811
  0.0902127   0.64431519  1.20982728  0.63108817  0.51907166  1.18415659
  0.58054762  0.54451657  0.

# Testing

In [14]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7515238095238095


In [15]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 10.656216382980347
training time: 24282.57847905159
test time: 469.44969415664673
total time: 24762.684389591217
